In [1]:
from torch.utils.data import  DataLoader,Dataset

In [2]:
import os
import pickle
import numpy as np
import torch

In [3]:
class dataset_withIDs(Dataset):
    """Adaptive dataset."""

    def __init__(self,db_path='',ids_path='',
                 
                 ):
        self.db_path=db_path       
        self.ids_path=ids_path
        
        pickle_in = open(self.ids_path,'rb')
        self.list_IDs= pickle.load(pickle_in)
        self.list_IDs=list(self.list_IDs)
        pickle_in.close()   
        
        
    def __len__(self):
        return int(len(self.list_IDs))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        id=self.list_IDs[idx]
     
        x=np.load(os.path.join(self.db_path,id+'.npy'))
        x=torch.from_numpy(x)
        x = (x.permute(2,0,1)).float()

     
        return id,x

In [18]:
val_data=dataset_withIDs(db_path='./Features/resnet50/val/',
                        ids_path='ids_val.pickle',
                 )
                 
print(f"Length of val Data : {len(val_data)}")

Length of val Data : 119


In [19]:
test_data=dataset_withIDs(db_path='./Features/resnet50/test/',
                        ids_path='ids_test.pickle',
                 )
                 
print(f"Length of test Data : {len(test_data)}")


Length of test Data : 253


In [20]:
from torch.utils.data import  DataLoader

In [21]:
test_dl = DataLoader(test_data, 1, shuffle = False)
val_dl = DataLoader(val_data, 1, shuffle = False)

In [8]:
from timesformer.models.vit import TimeSformer
import torch.nn as nn

In [9]:
model = TimeSformer(img_size=50,num_classes=1,patch_size=1,num_frames=20,attention_type='divided_space_time')

Pretrained model URL is invalid, using random initialization.


In [24]:
model.load_state_dict(torch.load('checkpoint.pth'))

<All keys matched successfully>

In [25]:
import time

In [26]:
device = torch.device("cuda")
if torch.cuda.is_available():
    model.cuda()

In [27]:
def step(inputs):
    data = inputs
    data=data.cuda()
    preds = model(data)
    return preds

In [28]:
part = 'val'

In [22]:
if part == 'val':
    dl = val_dl
if part == 'test':
    dl = test_dl


with open(f'output_{part}.txt', 'w') as f:
    with torch.no_grad():
        y_pred=[]
        ids=[]
        t = []
        for i,(id, batch) in enumerate(tqdm(dl)):
            t1=time.time()
            pred = step(batch)
            t2=time.time()
            t.append(t2-t1)
            y_pred.append(float(pred[0][0].cpu())*100)
            ids.append(id[0])
            line = f"{id[0]},{y_pred[-1]}"
            f.write(line)
            f.write('\n')


100%|███████████████████████████████████████████████████████████████████████████████| 119/119 [00:03<00:00, 37.64it/s]


In [23]:
np.mean(t)

0.02499722232337759